# Problem Statement

When testing complex REDCap scripts or functionalities, it is **essential** that one tests them in the Test project first before running/using them on the target project.

# Solution

This notebook exports data from the target project, then import them into the test project. The test project then will have the exact same records as the target project, effectively making them the same.

### **This notebook will wipe out all existing data from the Test project before the import. The existing data will still be recoverable through a JSON file, which can later be reimported back.** 

# Instructions
Currently, the API does not allow for importing instruments. Importing instruments has to be done manually.

1. Download all instrument zips from target project.
2. Download current test project's instrument zips if not relevant to target project.
3. Delete current instruments from test project.
4. Import the target project's instrument zips into test project.
5. Configure repeating instruments if needed.
6. Run this notebook.



# Code 
### 1. Setup target and test project

In [ ]:
import AMBRA_Backups
import json
import pandas as pd
from datetime import datetime

In [ ]:
# TARGET PROJECT
target_project_name = 'I3C DECADE - Image Tracking'
target_project = AMBRA_Backups.redcap_funcs.get_redcap_project(target_project_name)

# TEST PROJECT
test_project_name = 'Personal Test'
test_project = AMBRA_Backups.redcap_funcs.get_redcap_project(test_project_name)

Wipe out all existing data from `test_project`.

In [ ]:
record_column = 'record_id'
all_records = test_project.export_records(format_type='df', export_blank_for_gray_form_status=True)
all_records.reset_index(inplace=True)

# Get record IDs from Test
all_records_ids = []
if record_column in all_records:
   all_records_ids = all_records['record_id'].unique()

# Before deleting records, export all current records into a json file
all_records_json = test_project.export_records(export_blank_for_gray_form_status=True)
json_obj = json.dumps(all_records_json)

with open(f'{test_project_name}_{datetime.now()}.json', 'w') as f:
   f.write(json_obj)


['B00000610' 'B00001190' 'B00001393' 'B00002530' 'B00002722' 'B00002902'
 'B00003802' 'B00004032' 'B00004372' 'B00005031' 'B00005085' 'B00006816'
 'B00007194' 'B00007330' 'B00007489' 'B00008165' 'B00009607' 'B00012481'
 'B00012652' 'B00014636' 'B00014673' 'B00015059' 'B00015313' 'B00015584'
 'B00016512' 'B00019456' 'B00019680' 'B00020113' 'B00020620' 'B00024704'
 'B00025193' 'B00026988' 'B00027665' 'B00032112' 'B00033088' 'B00033991'
 'B00037422' 'B00037489' 'B00038493' 'B00039091' 'B00039215' 'B00039252'
 'B00043041' 'B00043850' 'B00043929' 'B00044776' 'B00044912' 'B00045577'
 'B00048260' 'B00048647' 'B00052882' 'B00054160' 'B00057509' 'B00057648'
 'B00058593' 'B00059976' 'B00061970' 'B00062564' 'B00062576' 'B00066403'
 'B00066485' 'B00066640' 'B00066850' 'B00066992' 'B00067074' 'B00068019'
 'B00068265' 'B00068384' 'B00069073' 'B00072912' 'B00073072' 'B00074416'
 'B00079632' 'B00080161' 'B00080800' 'B00081008' 'B00083281' 'B00086832'
 'B00089281' 'B00089393' 'B00090119' 'B00090864' 'B

In [ ]:
# Delete all records
if len(all_records_ids) > 0:
   test_project.delete_records(records=all_records_ids)

test_current_records = test_project.export_records(format_type='df', export_blank_for_gray_form_status=True)

if len(test_current_records) == 0:
   print(f'All records from {test_project_name} REDCap have been deleted')
else:
   raise ValueError(f'''Some records remain in {test_project_name} REDCap. current records:
         ---------
         {test_current_records}
         ''')

### 2. Copy records

In [5]:
'''
Get current fields from test_project to use as fields filter for
target_project's export records. 

This is needed, as when fields get deleted from a project, they 
will still show in the export_records() if there are values
in them. 

Since test_project does not have data yet, it would have only the
relevant fields and not the deleted fields.
'''
test_fields_dict = test_project.export_field_names()
test_fields = set()

for field in test_fields_dict:
   test_fields.add(field['original_field_name'])

target_records = target_project.export_records(
   fields=test_fields, 
   export_blank_for_gray_form_status=True
)
target_records

Define fields that need to be taken off. 

This is necessary in cases where options were defined but got deleted from a variable. They would still retain in `export_records` if there's data in them. This is a manual process. 

Instructions:

1. Set `delete_fields = []`. Run cell.
2. If an error is thrown, copy the fields mention in the error into 
`delete_fields`.
3. Run cell again.

In [6]:
delete_fields = [
   'sequences_acquired___ni',
   'sequences_collected___ni'
]

for record in target_records:
   for field in delete_fields:
      record.pop(field, None)

Import data into test_project

In [ ]:
test_project.import_records(target_records)

In [ ]:
# See if records imported
test_records_df = test_project.export_records(format_type='df', export_blank_for_gray_form_status=True)
target_records_df = target_project.export_records(format_type='df', export_blank_for_gray_form_status=True, fields=test_fields)

merged_df = pd.merge(test_records_df, target_records_df, how='outer', indicator=True)
if (len(merged_df) == len(test_records_df)) & (len(merged_df) == len(target_records_df)):
    print(f'''{test_project_name} imported data from {target_project_name} successfully''')
else:
    merged_df.to_excel(f'{test_project_name}_{target_project_name}_merged_{datetime.now()}.xlsx', header=True)
    raise ValueError(f'''Data discrepancies between {test_project_name} and {target_project_name}''')